이 노트는 Python 3.7로 작성됐습니다.

# What does *Pythonic* mean?.?

Python 입문하신 분들은 누구나 해봤을 질문

왜 collection.len()이 아니고 len(collection)이지?.?

*Pythonic* 한거는 뭐지?.?

In [16]:
arr = [1, 2, 3]
len(arr)

3

다들 위 코드가 돌아간다는 것은 알지만, 왜 어떻게 돌아가는거지?.?


**len**은 Python에서 취급하는 특수 함수

구체적인 예시:

In [17]:
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
deck = FrenchDeck()
print(len(deck))

52


**len** 함수가 실행하는 것은 Python 내부의 \_\_len\_\_함수

혹시 직접 만든 함수들도 비슷한 방식으로 호출할 수 있을까?.?

In [18]:
import traceback

class TheoryTest:
    def __init__(self):
        self.items = [1, 2, 3, 4]
        
    def __len__(self):
        return len(self.items)
    
    def __testtheory__(self):
        print("Tested?.?")
    
test = TheoryTest()
print(len(test))
try:
    testtheory(test)# 과연 될까?.?
except Exception as e:
    tb = traceback.format_exc()# 오류 나지만 돌아가게 하기 위한 부분
    print("\n" + tb)

4

Traceback (most recent call last):
  File "<ipython-input-18-7aaf425ee2e8>", line 16, in <module>
    testtheory(test)# 과연 될까?.?
NameError: name 'testtheory' is not defined



역시는 역시 안된다..

하지만..?

In [19]:
def testtheory(test):
    test.__testtheory__()

# 된다는 것을 보여주기 위한 예시일 뿐
# 하지는 말자...
testtheory(test)

Tested?.?


### 특수 함수 (Special Method)는 어떻게 사용되는가?.?

Python interpreter에서 예약된 함수들을 활용해서, object model 내부적으로 구현된 함수를 실행하는것.

기본 데이터형들에 대해서는 Python에서 조금 더 최적화된 방법으로 값을 뽑아내는 경우가 있기도 하고, 직접 호출하는것은 지양 하는것이 일반적이다.

**결론**: 편하게 만들어준 것은 편하게 쓰자

<br />

Python 지지자들이 내세우는 주장 중 하나

**Python의 가장 큰 장점중 하나는 일관성이다.**

In [20]:
arr = [1, 2, 3, 4, 5]
print(arr.__len__()) # 하지말고
print(len(arr)) # 굿

5
5


특수 함수들은 한정되어 있고, 편의를 위해서 만들어졌다는 것이 일반적인 시선이다. 그 동안 Python에서 *어떻게 구현된거지?.?* 했던 마법 같은 기능들은 배후에 특수 함수가 있을 가능성이 높다.

일부 마법을 파헤치기 위해, 몇가지 예시를 짚고 넘어가 보자

#### 1. repr 함수, str 함수

```pythoh
>>> a = Vector(3, 5)
>>> print(a)
Vector(3, 5)
```

이런식으로 프린트를 했을때 객체 형태로 보기에 이쁘게 나오는 것을 어딘가에서 본적 있을거다.

In [31]:
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        
    def __repr__(self): # representation의 약자인것 같다
        return 'Vector(%r, %r)' % (self.x, self.y)
    
    def __str__(self): # java의 toString과 비슷
        return f"(x={self.x} y={self.y})"
x = Vector(3, 5)
print(repr(x))
print(x)

Vector(3, 5)
(x=3 y=5)


**repr** 특수 함수를 통해서 구현돼 있었던 거였다!.!

repr 함수는 일번적으로 공식적인 객체를 나타내는 수단으로 사용된다 (보통 그 객체를 생성하기 위해 사용된 생성자문)

디버깅 용도로 주로 사용

**str** 함수는 실제 유저에게 보여줄 목적으로 생성하는 문자열 형태를 만들기 위해 구현


#### 2. getitem 함수

리스트형 객체에서 사용

In [68]:
import collections
import random
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def shuffle(self):
        random.shuffle(self._cards)
        
deck = FrenchDeck()
deck.shuffle()
deck[0]

for card in deck[:10]:
    print(card)

Card(rank='9', suit='spades')
Card(rank='J', suit='hearts')
Card(rank='J', suit='spades')
Card(rank='Q', suit='clubs')
Card(rank='4', suit='hearts')
Card(rank='3', suit='diamonds')
Card(rank='7', suit='diamonds')
Card(rank='5', suit='clubs')
Card(rank='6', suit='diamonds')
Card(rank='K', suit='clubs')


> ## *practicality beats purity*